In [1]:
import pandas as pd
pd.set_option('display.max_columns', 500)

import warnings
warnings.simplefilter('ignore')

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg' 
from pylab import rcParams
rcParams['figure.figsize'] = 16, 10

In [37]:
support_tickets = pd.read_csv('support_tickets.csv')
support_tickets.head()

,user_id,ticket_category,ticket_subcategory,current_state,request_id,activity_start_dt,fact_reaction_dt
0,370 518 212,Технические вопросы,Доступ к сайту,Решено,045be6ac-d532-4c20-b1c9-fb82c1737464,2015-11-26 22:56:00,2015-12-03 12:11:00
1,2 138 799,Технические вопросы,Доступ к сайту,Решено,8ebbb8c6-7abb-4fad-a83b-35a71a83a186,2015-12-18 08:52:00,2015-12-23 06:40:00
2,2 033 101,Технические вопросы,Доступ к сайту,Решено,ee303751-4a21-4544-a4c3-56924c088028,2015-12-16 19:06:00,2015-12-26 07:46:00
3,177 122 000 003,Мобильный Avito,Android приложение,Решено,bb989fcb-277a-41bd-aaa2-f131333b629e,2015-12-01 17:08:00,2015-12-09 11:19:00
4,37 340 500 064,Технические вопросы,Доступ к сайту,Решено,e742d172-0ef1-47d8-b862-25be082d8bc0,2015-12-14 10:08:00,2015-12-15 16:14:00


In [81]:
users_evaluation = pd.read_csv('users_evaluation_of_satisfaction.csv')
users_evaluation.head()

,request_id,result_mentioned_by_user
0,09e34912-b4df-4e91-b3e0-35b40fe6b9b8,Не удовлетворительно
1,76099bef-7ab3-4a01-96c3-4c06f4032a7e,Не удовлетворительно
2,70acfed6-8897-4e38-a407-65150e7a4395,Удовлетворительно
3,64c7a6c8-8bfe-4f35-b662-9091b3472e6f,Не удовлетворительно
4,b08d9414-7167-4f7f-9837-fe6e535f4fb6,Не удовлетворительно


In [21]:
new_items = pd.read_csv('new_items_by_support_users.csv')
new_items.head()

,user_id,user_registration_time,user_first_listing_date,item_id,item_starttime,item_category,item_subcategory
0,386 341 477,2013-05-27 14:40:09,2013-05-27 14:40:09,8 523 500 011,2014-01-01 02:22:54,Транспорт,Автомобили
1,6 790 750 024,2013-12-14 23:05:15,2013-12-14 23:27:04,8 525 000 013,2014-01-01 02:45:56,Транспорт,Запчасти и аксессуары
2,2 755 740,2011-04-14 17:54:40,2011-04-14 18:05:27,8 527 000 043,2014-01-01 03:29:44,Транспорт,Автомобили
3,506 779,2011-08-14 06:19:11,2011-08-14 06:50:04,8 530 000 039,2014-01-01 04:34:51,Личные вещи,Красота и здоровье
4,766 896,2013-05-17 11:32:13,2013-06-05 06:05:09,8 534 500 187,2014-01-01 06:07:58,Недвижимость,"Дома, дачи, коттеджи"


## Вычислим как влияет время ответа техподдержки на следующие параметры:
* Оценка
* Количество объявлений после обращения


In [39]:
support_tickets['activity_start_dt'] = pd.to_datetime(support_tickets['activity_start_dt'])
support_tickets['fact_reaction_dt'] = pd.to_datetime(support_tickets['fact_reaction_dt'])

In [40]:
support_tickets['duration'] = support_tickets['fact_reaction_dt'] - support_tickets['activity_start_dt']

In [82]:
users_evaluation['result_mentioned_by_user'] = users_evaluation['result_mentioned_by_user'].map(
    {'Отлично': 5, 'Хорошо': 4, 'Нейтрально': 3, 'Удовлетворительно': 2, 'Не удовлетворительно': 1})

In [94]:
users_evaluation = users_evaluation.groupby('request_id', as_index=False).mean()

In [99]:
support_tickets_w_ev = pd.merge(support_tickets, users_evaluation, how='inner', on='request_id')

In [125]:
support_tickets_w_ev.drop(support_tickets_w_ev.loc[
    support_tickets_w_ev['duration'] <= pd.to_timedelta(0)].index, inplace=True)

In [118]:
support_tickets_w_ev['result_mentioned_by_user'].plot(kind='duration')

ValueError: 'duration' is not a valid plot kind

In [146]:
support_tickets_w_ev['duration'].apply(total_seconds())

NameError: name 'total_seconds' is not defined